<a href="https://colab.research.google.com/github/lluisdn/TFM/blob/Additional/Variational_autoencoder_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Variational autoencoder**


#*Imports*

In [ ]:
import numpy as np
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.datasets import mnist

#*Dataset*

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

# Preprocessing
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


11490434/11490434 [==============================] - 1s 0us/step


#*Autoencoder*

784 --> 512 --> 256 --> (z_mean, z_var) --> 256 --> 512 -->784

In [ ]:
input_shape = (784, )
latent_dim = 2

# Encoder
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(512, activation='relu')(inputs)
x = Dense(256, activation='relu')(x)

# Mean and variance layer
z_mean = Dense(latent_dim, name='z_mean')(x)
z_var = Dense(latent_dim, name='z_var')(x)


# Sampling function
def sampling(z_mean, z_var):
    epsilon = np.random.normal(loc=0., scale=1.)
    return z_mean + z_var * epsilon

# Sample from the latent space
z = sampling(z_mean, z_var)

# Decoder
decoder_inputs = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(256, activation='relu')(decoder_inputs)
x = Dense(512, activation='relu')(x)
outputs = Dense(784, activation='sigmoid')(x)

# Encoder model
encoder = Model(inputs, [z_mean, z_var, z], name='encoder')

#Decoder model
decoder = Model(decoder_inputs, outputs, name='decoder')

# VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

#*Loss function*

ELBO... to do

#*Compile*

In [ ]:
vae.compile(optimizer='adam', loss='mean_squared_error')

#*Summary*

In [ ]:
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               534276    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               534544    
                                                                 
Total params: 1,068,820
Trainable params: 1,068,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 784)]        0           []                               
                                                                                                  
 dense_10 (Dense)               (None, 512)          401920      ['encoder_input[0][0]']          
                                                                                                  
 dense_11 (Dense)               (None, 256)          131328      ['dense_10[0][0]']               
                                                                                                  
 z_var (Dense)                  (None, 2)            514         ['dense_11[0][0]']               
                                                                                            

In [ ]:
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_12 (Dense)            (None, 256)               768       
                                                                 
 dense_13 (Dense)            (None, 512)               131584    
                                                                 
 dense_14 (Dense)            (None, 784)               402192    
                                                                 
Total params: 534,544
Trainable params: 534,544
Non-trainable params: 0
_________________________________________________________________


#*Train*


In [ ]:
vae.fit(x_train, x_train, epochs=5)

Epoch 1/5


TypeError: ignored

#*Predictions*